# Tune the hyperparameters 
Using keras tuner, solve for the optimal hyperparameters 

In [1]:
import os # Used to make a directory for saving results 
import xarray as xr 
import pandas as pd 
import numpy as np 
import keras_tuner as kt 
from tqdm.keras import TqdmCallback # Progress bar during fitting 
import tensorflow as tf 
from sklearn.utils.class_weight import compute_class_weight

# Import helper functions 
import sys
sys.path.insert(0, '../../utils')
from cnn_utils import (
    onehot, 
    build_kt_tuning_model, 
)

Using TensorFlow backend


2023-10-26 03:19:18.123563: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Class 0: no extreme precip 
# Class 1: extreme precip
classes = [0,1]

# Columns to use for labels vs. features 
labels = "precip_classes"
features_list = ["slp_anom","hgt_detrended_anom"]

# Reference directories 
data_dir = "../../data/input_data_preprocessed/" # Input data

In [3]:
# Read csv as pandas DataFrame object
x_train_ds = xr.open_dataset(data_dir+"training/training_features.nc")
y_train_df = pd.read_csv(data_dir+"training/training_labels.csv", index_col=False)

x_val_ds = xr.open_dataset(data_dir+"validation/validation_features.nc")
y_val_df = pd.read_csv(data_dir+"validation/validation_labels.csv", index_col=False)

x_test_ds = xr.open_dataset(data_dir+"testing/testing_features.nc")
y_test_df = pd.read_csv(data_dir+"testing/testing_labels.csv", index_col=False)

In [4]:
# Convert to numpy arrays
x_train = x_train_ds.to_array().transpose("time","lat","lon","variable").values
y_train = y_train_df[labels].values

x_val = x_val_ds.to_array().transpose("time","lat","lon","variable").values
y_val = y_val_df[labels].values

x_test = x_test_ds.to_array().transpose("time","lat","lon","variable").values
y_test = y_test_df[labels].values

In [5]:
# Compute class weights 
class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=np.array(classes), 
    y=y_train
)
class_weights = {classes[0]:class_weights[0], classes[1]:class_weights[1]}
print(class_weights)

{0: 0.5273766893843133, 1: 9.631856540084389}


In [6]:
# Do one hot encoding 
y_train_onehot = onehot(y_train)
y_val_onehot = onehot(y_val)
y_test_onehot = onehot(y_test)

In [7]:
# Model settings 
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True
)
settings = {
    "callbacks": early_stopping,
    "class_weights": class_weights, 
    "epochs": 500 # Some big number here. Shouldn't matter too much because we implement early stopping
}

### Tune the model parameters 
Reference pages: 
- https://keras.io/guides/keras_tuner/getting_started/
- https://neptune.ai/blog/keras-tuner-tuning-hyperparameters-deep-learning-model

Note to self: KT objective should be the same as the LOSS. Need to pick one. 

In [8]:
num_trials = 30
tuner = kt.RandomSearch( # kt.GridSearch will go through all options 
    build_kt_tuning_model,
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=num_trials,
    overwrite=True, 
    project_name='hyperband_tuner'
)
tuner.search_space_summary()
tuner.search(
    x_train, y_train_onehot,
    validation_data=(x_val, y_val_onehot), 
    epochs=settings["epochs"],  
    class_weight=settings["class_weights"], 
    callbacks=[settings["callbacks"]], 
)

Trial 30 Complete [00h 01m 38s]
val_loss: 0.39463692903518677

Best val_loss So Far: 0.32365894317626953
Total elapsed time: 00h 39m 48s


In [10]:
tuner.results_summary(num_trials=5)

Results summary
Results in ./hyperband_tuner
Showing 5 best trials
Objective(name="val_loss", direction="min")

Trial 22 summary
Hyperparameters:
learning_rate: 0.0005
activity_reg_factor: 0.0001
num_conv_filters: 24
num_dense_neurons: 8
num_dense_layers: 3
random_seed: 70
Score: 0.32365894317626953

Trial 16 summary
Hyperparameters:
learning_rate: 0.0005
activity_reg_factor: 0.0001
num_conv_filters: 8
num_dense_neurons: 24
num_dense_layers: 2
random_seed: 170
Score: 0.3691146969795227

Trial 20 summary
Hyperparameters:
learning_rate: 0.0025
activity_reg_factor: 0.0001
num_conv_filters: 8
num_dense_neurons: 32
num_dense_layers: 3
random_seed: 110
Score: 0.3759380578994751

Trial 28 summary
Hyperparameters:
learning_rate: 0.0005
activity_reg_factor: 0.0025
num_conv_filters: 24
num_dense_neurons: 8
num_dense_layers: 3
random_seed: 200
Score: 0.39374178647994995

Trial 29 summary
Hyperparameters:
learning_rate: 0.0005
activity_reg_factor: 0.0005
num_conv_filters: 24
num_dense_neurons: 16
